In [2]:
import pymysql as m

con = m.connect(host = '54.164.15.159', database = 'team06-antifragile-db',
                user = 'admin', password = 'antifragile1234', charset = 'utf8')

OperationalError: (2003, "Can't connect to MySQL server on '54.164.15.159' (timed out)")

In [ ]:
cur = con.cursor()